In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
import operator
from itertools import combinations_with_replacement
from scipy.special import comb
from random import randrange
import random
from numpy import nan

import warnings
warnings.filterwarnings('ignore')

In [2]:
#1
def polynomialFeatures(X,degree = 1):
    poly_comb=np.zeros((X.shape[0],comb(X.shape[1]+degree,degree).astype(int)-1))
    for i in range(X.shape[0]):
        temp_list = []
        for j in range(degree):
            temp_arr = np.array(list(combinations_with_replacement(X[i],j+1)))
            temp_list.append(poly_helper(temp_arr))
        poly_comb[i]=np.concatenate(temp_list, axis=None)
    return poly_comb

def poly_helper(row):
    size,ele = row.shape
    if ele == 1:
        return row
    arr = np.zeros((size))
    for i in range(size):
        temp=1
        for j in range(ele):
            temp*=row[i][j]
        arr[i]=temp
    return arr.tolist()

In [3]:
#2
def mse(Y_true,Y_pred):
    return np.mean((np.square(Y_true - Y_pred)))

In [4]:
def standardize(X):
    return (X-np.mean(X,axis = 0))/np.std(X,axis=0)

In [5]:
def partition(data,target,t):
    train_data, test_data = np.split(data, [int((1-t)*len(data))])
    target_train,target_test = np.split(target, [int((1-t)*len(target))])
    return train_data,test_data,target_train,target_test

In [6]:
#3
def learning_curve(model, X, Y, cv, train_size=1, learning_rate=0.01,
                   epochs=1000, tol=None, regularizer=None, lambd=0.0, **kwargs):
    model_arg = {"learning _rate" : learning_rate,"epochs":epochs,"tol":tol,"regularizer":regularizer,"lambd":lambd}
    if not isinstance(train_size, int):
        train_size = int (X.shape[0]*train_size)
        t_size = train_size
    else:
        tsize = train_size
    flag = 1
    train_scores = []
    val_scores = []
    while flag:
        train,val = sfolds_cross_val(cv,X[:train_size],Y[:train_size],model,model_arg)
        train_scores.append(np.sqrt(train))
        val_scores.append(np.sqrt(val))
        train_size += t_size
        if train_size > X.shape[0]:
            flag = 0
            train,val = sfolds_cross_val(cv,X[:train_size],Y[:train_size],model,model_arg)
            train_scores.append(np.sqrt((train)))
            val_scores.append(np.sqrt(val))
    return train_scores,val_scores

In [7]:
#4
def plot_polynomial_model_complexity(model, X, Y, cv, maxPolynomialDegree,
                                     learning_rate=0.01, epochs=1000, tol=None, regularizer=None, lambd=0.0,
                                     **kwargs):
    train_scores=[]
    val_scores=[]
    for i in range(maxPolynomialDegree):
        X_aug = polynomialFeatures(X,i+1)
        X_aug = standardize(X_aug)
        if i < 3:
            model_arg = {"learning_rate" : learning_rate,"epochs":epochs,"tol":tol,"regularizer":regularizer,"lambd":lambd}
            train,val = sfolds_cross_val(cv,X_aug,Y,model,model_arg)
            train_scores.append(np.sqrt(train))
            val_scores.append(np.sqrt(val))
        else:
            if learning_rate > 0.001:
                model_arg = {"learning_rate" : 0.001,"epochs":epochs,"tol":tol,"regularizer":regularizer,"lambd":lambd}
            else:
                model_arg = {"learning_rate" : learning_rate,"epochs":epochs,"tol":tol,"regularizer":regularizer,"lambd":lambd}
            train,val = sfolds_cross_val(cv,X_aug,Y,model,model_arg)
            train_scores.append(np.sqrt(train))
            val_scores.append(np.sqrt(val))
    plt.figure(figsize=(10, 6))
    plt.plot([j for j in range (1,maxPolynomialDegree+1)], (train_scores), "r-+", linewidth=3, label="Training Score")
    plt.plot([j for j in range (1,maxPolynomialDegree+1)], (val_scores), "b-", linewidth=2, label="Cross-validation Score")
    plt.legend(loc="best", fontsize=18)   
    plt.xlabel("Training set size", fontsize=18) 
    plt.ylabel("RMSE", fontsize=14) 
    plt.title(" (Polynomial Model)")
    plt.show()
    print("Optimal Degree: ",np.argmin(val_scores)+1)

In [8]:
#5
class Linear_Regression:
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,
             lambd=0.0, **kwargs):
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.tol=tol
        self.regularizer=regularizer
        self.lambd=lambd
        m = X.shape[0]
        ones = np.ones([X.shape[0],1])
        X = np.concatenate((ones,X),axis=1)
        theta_hat = np.zeros(X.shape[1])
        previous_error = mse(Y,np.dot(X,theta_hat)-Y)
        for i in range(epochs):
            if regularizer == 'l1' :
                theta_hat -= (learning_rate/m) * np.dot(X.T,np.dot(X,theta_hat)-Y) - (learning_rate*lambd*np.sign(theta_hat))/m
            elif regularizer == 'l2':
                theta_hat -= (learning_rate/m) * np.dot(X.T,np.dot(X,theta_hat)-Y) - (learning_rate*lambd*theta_hat)/m
            else:
                theta_hat -= (learning_rate/m) * np.dot(X.T,np.dot(X,theta_hat)-Y)
            error = mse(Y,np.dot(X,theta_hat)-Y)
            if (tol is not None):
                if (error > previous_error - tol) :
                    self.theta_hat = theta_hat
                    break
            else:
                previous_error = error
        self.theta_hat = theta_hat
        
    def predict(self,X):
        ones = np.ones([X.shape[0],1])
        X = np.concatenate((ones,X),axis=1)
        return np.dot(X,self.theta_hat)
    
    def __int__(self):
        self

In [9]:
#extra credits 
class Stochastic_Regression:
    def fit(self, X, Y, learning_rate=0.01, epochs=100, tol=None, regularizer=None,
             lambd=0.0, **kwargs):
        #print(learning_rate)
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.tol=tol
        self.regularizer=regularizer
        self.lambd=lambd
        m = X.shape[0]
        ones = np.ones([X.shape[0],1])
        X = np.concatenate((ones,X),axis=1)
        theta_hat = np.zeros(X.shape[1])
        previous_error = mse(Y,np.dot(X,theta_hat)-Y)
        decay = learning_rate/epochs
        learn = learning_rate
        for i in range(epochs):
            j = list(range(1,X.shape[0])) 
            if regularizer == None :
                theta_hat -= (learning_rate/m) * np.dot(X[j].T,np.dot(X[j],theta_hat)-Y[j])
            elif regularizer == 'l2':
                theta_hat -= (learning_rate/m) * np.dot(X[j].T,np.dot(X[j],theta_hat)-Y[j]) - (learning_rate*lambd*theta_hat)/m
            else:
                theta_hat -= (learning_rate/m) * np.dot(X[j].T,np.dot(X[j],theta_hat)-Y[j]) - (learning_rate*lambd*np.sign(theta_hat))/m
                
            error = mse(Y,np.dot(X,theta_hat)-Y)
            learning_rate =learn * 1/(1+decay*i)
            if (tol is not None):
                if (error > previous_error - tol) :
                    self.theta_hat = theta_hat
                    break
            else:
                previous_error = error
        #print(learning_rate)
        self.theta_hat = theta_hat
    
    def predict(self,X):
        ones = np.ones([X.shape[0],1])
        X = np.concatenate((ones,X),axis=1)
        return np.dot(X,self.theta_hat)
    
    def __int__(self):
        self

In [10]:
def sfolds(folds,data,labels,model,model_args,err_func):
    values={}
    X_folds = np.array_split(data,folds)
    Y_folds = np.array_split(labels,folds)
    predicted = []
    error=[]
    for X,Y in zip(X_folds,Y_folds):
        model.fit(X,np.array(Y),**model_args)
        predicted.append(model.predict(X))
        error.append(err_func(Y,model.predict(X)))
    pred_values=np.concatenate(np.asarray((predicted)),axis=0)
    return pred_values,np.array(error)

In [11]:
def sfolds_cross_val(folds,data,labels,model,model_args):
    train_scores =[]
    val_scores = []
    for i in range(folds):
        X_folds = np.array_split(data,folds)
        Y_folds = np.array_split(labels,folds)
        X_test = np.array(X_folds.pop(i))
        Y_test = np.array(Y_folds.pop(i))
        X_train = np.vstack(X_folds)
        Y_train = np.hstack(Y_folds)
        model.fit(X_train,Y_train,**model_args)
        train_scores.append(mse(Y_train,model.predict(X_train)))
        val_scores.append(mse(Y_test,model.predict(X_test)))
    return np.mean(train_scores),np.mean(val_scores)

In [12]:
#6
df = pd.read_csv('winequality-red.csv',sep=";")
df.head()


FileNotFoundError: [Errno 2] File b'winequality-red.csv' does not exist: b'winequality-red.csv'

In [ ]:
#7
df.describe()

In [ ]:
#7
df.corr()

In [ ]:
#8
df = df.sample(frac=1) 

In [ ]:
#9
%matplotlib inline

import seaborn as sns
from scipy import stats
def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.1, .6), xycoords=ax.transAxes,
               size = 24)
    
cmap = sns.cubehelix_palette(light=1, dark = 0.1,
                             hue = 0.5, as_cmap=True)

sns.set_context(font_scale=2)
g = sns.PairGrid(df)
g.map_upper(plt.scatter, s=10, color = 'red')


g.map_diag(sns.distplot, kde=False, color = 'red')


g.map_lower(sns.kdeplot, cmap = cmap)
g.map_lower(corrfunc);

In [ ]:
np.abs(df.corr()['quality']).sort_values(ascending=False)

In [ ]:
df=df.drop(columns =['free sulfur dioxide','citric acid'])

In [ ]:
X = df.drop(columns="quality").values
Y=df['quality']

In [ ]:
lambd = [1.0, 0, 0.1, 0.01, 0.001, 0.0001]
learning_rate = [0.1, 0.01, 0.001, 0.0001]
regularizer = ['l1', 'l2']
X_train,X_test,Y_train,Y_test = partition(standardize(X),Y,0.2)

In [ ]:
#10
min_mse = 100
min_key = ""
params = {}
for l in lambd:
    for l_r in learning_rate:
        for reg in regularizer:
            key = 'lambd: '+str(l)+' learning_rate: '+str(l_r)+ " regularizer: "+reg
            model_args = {"learning_rate" : l_r,"epochs":1000,"tol":None,"regularizer":reg,"lambd":l}
            params[key]=sfolds(10,X_train,Y_train,Linear_Regression(),model_args,mse)
            if np.mean(params[key][1]) < min_mse:
                min_mse=np.mean(params[key][1])
                min_key=key

In [ ]:
print("Minimum Mse error was for :",min_key)
print(min_mse)

In [ ]:
#11
optimal_params = min_key.split(' ')
optimal_args = {"learning_rate" : float (optimal_params[3]),"epochs":1000,"tol":0.001,"regularizer":optimal_params[5],"lambd": float (optimal_params[1])}

In [ ]:
#11
y_test_preds,y_test_error = sfolds(10,X_test,Y_test,Linear_Regression(),optimal_args,mse)
print("MSE for y_test :",np.mean(y_test_error))

In [ ]:
#12
cv = 10
train_scores = []
val_scores = []
train_scores,val_scores = learning_curve(Linear_Regression(),X_train,Y_train,cv,0.01,**optimal_args)


In [ ]:

plt.figure(figsize=(10, 6))
plt.plot([j*X.shape[0]/len(train_scores) for j in range(len(train_scores))],train_scores, "r-+", linewidth=3, label="Training Score")
plt.plot([j*X.shape[0]/len(train_scores) for j in range(len(train_scores))],val_scores, "b-", linewidth=2, label="Cross-validation Score")
plt.legend(loc="best", fontsize=18)   
plt.xlabel("Training set size", fontsize=18) 
plt.ylabel("RMSE", fontsize=14) 
plt.title("Learning Curve (Linear Model)")
plt.show()

In [ ]:
#13
X_train,X_test,Y_train,Y_test = partition(X,Y,0.2)
X_aug = polynomialFeatures(X_train,3)
X_aug = standardize(X_aug)
min_mse_aug = 100
min_key_aug = ""
X_aug_params = {}

for l in lambd:
    for l_r in learning_rate:
        for reg in regularizer:
            key = 'lambd: '+str(l)+' learning_rate: '+str(l_r)+ " regularizer: "+reg
            model_args = {"learning_rate" : l_r,"epochs":1000,"tol":None,"regularizer":reg,"lambd":l}
            X_aug_params[key]=sfolds(10,X_aug,Y_train,Linear_Regression(),model_args,mse)
            if np.mean(X_aug_params[key][1]) < min_mse_aug:
                min_mse_aug=np.mean(X_aug_params[key][1])
                min_key_aug=key

In [ ]:
#13
print("Minimum Mse error was for :",min_key_aug)
print(min_mse_aug)

In [ ]:
#13
cv = 10
train_scores_aug = []
val_scores_aug = []
train_scores_aug,val_scores_aug = learning_curve(Linear_Regression(),X_aug,Y_train,cv,0.01,**optimal_args_aug)


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot([j*X.shape[0]/len(train_scores_aug) for j in range(len(train_scores_aug))],train_scores_aug, "r-+", linewidth=3, label="Training Score")
plt.plot([j*X.shape[0]/len(train_scores_aug) for j in range(len(val_scores_aug))],val_scores_aug, "b-", linewidth=2, label="Cross-validation Score")
plt.legend(loc="best", fontsize=18)   
plt.xlabel("Training set size", fontsize=18) 
plt.ylabel("RMSE", fontsize=14) 
plt.title("Learning Curve (Polynomial Model)")
plt.show()

In [ ]:
#14 extra credits
maximum_degree = 5
X_train,X_test,Y_train,Y_test = partition(X,Y,0.2)
plot_polynomial_model_complexity(Linear_Regression(),X_train,Y_train,10,maximum_degree)

In [ ]:
#15 extra credits
X_train,X_test,Y_train,Y_test = partition(standardize(X),Y,0.2)
min_mse_sgd = 100
min_key_sgd = ""
params_sgd = {}
cv=10
for l in lambd:
    for l_r in learning_rate:
        for reg in regularizer:
            key = 'lambd: '+str(l)+' learning_rate: '+str(l_r)+ " regularizer: "+reg
            model_args = {"learning_rate" : l_r,"epochs":1000,"tol":0.001,"regularizer":reg,"lambd":l}
            params_sgd[key]=sfolds_cross_val(10,X_train,Y_train,Stochastic_Regression(),model_args)
            if np.mean(params_sgd[key][1]) < min_mse_sgd:
                min_mse_sgd=np.mean(params_sgd[key][1])
                min_key_sgd=key

In [ ]:
print("Minimum Mse error was for :",min_key_sgd)
print(min_mse_sgd)

In [ ]:
optimal_params_sgd = min_key_sgd.split(' ')
optimal_args_sgd = {"learning_rate" : float (optimal_params_sgd[3]),"epochs":10000,"tol":0.001,"regularizer":optimal_params_sgd[5],"lambd": float (optimal_params_sgd[1])}


In [ ]:
y_test_preds,y_test_error = sfolds(10,standardize(X_test),Y_test,Stochastic_Regression(),optimal_args_sgd,mse)

print("MSE for y_test :",np.mean(y_test_error))